<h1><center>LSTM Language Detection</center></h1>
<h3><center>CSE 6240 - Websearch and Text Mining</center></h3>

<h7><center>Adam Lieberman, Garrett Mallory, Ravish Chawla</center></h7>
<h7><center>April 25, 2017</center></h7>

1) make 80/20 split on end and frn languages. (lower case all letters)
2) create two lstm models (size 128)
3) train 5 epochs, 0.2 validation split on the training samples 
4) 
4) train on 100 examples of 5 character strings from text with the right label.

pass new word through lstm to get log probability. then do generative log ratio test. y = sign(pe/pj -1) to get the label. Keep scores for ROC.

to get log probability, need p(t), p(r|t), p(u|tr) ... etc. Do this by passing in t, then tr then tru .... This gives you the most likely next character distribution so index into that to get the prob of "U" from "tr".

pass both sets of 100 test (all 200) through both networks for the probability part


In [3]:
from __future__ import print_function
import sys
import random
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

In [41]:
english_text = open('eng.txt').read().lower()
french_text = open('frn.txt').read().lower()

print('English corpus length:', len(english_text))
print('French corpus length:', len(french_text))

English corpus length: 10746
French corpus length: 12009
universal declaration of human rights
preamble whereas recognition of the inherent dignity and of the equal and inalienable rights of all members of the human family is the foundation of freedom, justice and peace in the world, whereas disregard and contempt for human rights have resulted in barbarous acts which have outraged the conscience of mankind, and the advent of a world in which human beings shall enjoy freedom of speech and belief and freedom from fear and want has been proclaimed as the highest aspiration of the common people, whereas it is essential, if man is not to be compelled to have recourse, as a last resort, to rebellion against tyranny and oppression, that human rights should be protected by the rule of law, whereas it is essential to promote the development of friendly relations between nations, whereas the peoples of the united nations have in the charter reaffirmed their faith in fundamental human rights, in

In [37]:
english_chars = sorted(list(set(english_text)))
french_chars = sorted(list(set(french_text)))

english_char_map = dict((c, i) for i, c in enumerate(english_chars))
french_char_map = dict((c, i) for i, c in enumerate(french_chars))

english_char_map_inverse = dict((i, c) for i, c in enumerate(english_chars))
french_char_map_inverse = dict((i, c) for i, c in enumerate(french_chars))

In [50]:
print('English character count:', len(english_chars))
print('French character count:', len(french_chars))
for c in french_chars:
    if c not in english_chars:
        print("note present:", c)

English character count: 43
French character count: 41
note present: '


In [39]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3

english_sentences = []
english_next_chars = []
for i in range(0, len(english_text) - maxlen, step):
    english_sentences.append(english_text[i: i + maxlen])
    english_next_chars.append(english_text[i + maxlen])

french_sentences = []
french_next_chars = []
for i in range(0, len(french_text) - maxlen, step):
    french_sentences.append(french_text[i: i + maxlen])
    french_next_chars.append(french_text[i + maxlen])
    
print('nb English sequences:', len(english_sentences))
print('nb French sequences:', len(french_sentences))

nb English sequences: 3569
nb French sequences: 3990


In [40]:
print('Vectorization...')
english_x = np.zeros((len(english_sentences), maxlen, len(english_chars)), dtype=np.bool)
english_y = np.zeros((len(english_sentences), len(english_chars)), dtype=np.bool)
for i, sentence in enumerate(english_sentences):
    for t, char in enumerate(sentence):
        english_x[i, t, english_char_map[char]] = 1
    english_y[i, english_char_map[english_next_chars[i]]] = 1
    
    
french_x = np.zeros((len(french_sentences), maxlen, len(french_chars)), dtype=np.bool)
french_y = np.zeros((len(french_sentences), len(french_chars)), dtype=np.bool)
for i, sentence in enumerate(french_sentences):
    for t, char in enumerate(sentence):
        french_x[i, t, french_char_map[char]] = 1
    french_y[i, french_char_map[french_next_chars[i]]] = 1

print("Finished!")

Vectorization...
Finished!


In [65]:
# build the model: a single LSTM
def build_model(chars):
    print('Build model...')
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, 43)))
    model.add(Dense(43))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

print (len(english_chars))
english_model = build_model(english_chars)
english_model.fit(english_x, english_y, batch_size=128, epochs=5)


43
Build model...
Epoch 1/5
3569/3569 [==============================] - 2s - loss: 3.0919     
Epoch 2/5
3569/3569 [==============================] - 2s - loss: 2.7503     
Epoch 3/5
3569/3569 [==============================] - 2s - loss: 2.4440     
Epoch 4/5
3569/3569 [==============================] - 2s - loss: 2.2529     
Epoch 5/5
3569/3569 [==============================] - 2s - loss: 2.0993     


In [18]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [56]:
def train_model(model, text, chars, char_map, char_map_inverse, x, y):
    # train the model, output generated text after each iteration
    for iteration in range(1, 60):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x, y,
                  batch_size=128,
                  epochs=5)

        start_index = random.randint(0, len(text) - maxlen - 1)

        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x[0, t, char_map[char]] = 1.

                preds = model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = char_map_inverse[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [64]:
print (english_model.summary)
print (len(english_text))
print (english_x.shape)
print (english_y.shape)

english_model.fit(english_x, english_y, batch_size=128, epochs=5)


#train_model(english_model, english_text, english_chars, english_char_map, english_char_map_inverse, english_x, english_y)

<bound method Container.summary of <keras.models.Sequential object at 0x7f489075e4e0>>
10746
(3569, 40, 43)
(3569, 43)
Epoch 1/5
3569/3569 [==============================] - 2s - loss: 3.0911     
Epoch 2/5
3569/3569 [==============================] - 2s - loss: 2.7563     
Epoch 3/5
3569/3569 [==============================] - 2s - loss: 2.4389     
Epoch 4/5
3569/3569 [==============================] - 2s - loss: 2.2112     
Epoch 5/5
3569/3569 [==============================] - 2s - loss: 2.0357     
